In [22]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt

DATA_FOLDER = "/Users/karunya/Documents/EPFL/ADA/ADA_Group_Tota_Pasari_Prelot/04 - Applied ML/"

In [24]:
# import csv data into a dataframe
file = glob.glob(DATA_FOLDER + "lalonde.csv")
df = pd.read_csv(file[0])
df.head()

,id,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
0,NSW1,1,37,11,1,0,1,1,0.0,0.0,9930.0460
1,NSW2,1,22,9,0,1,0,1,0.0,0.0,3595.8940
2,NSW3,1,30,12,1,0,0,0,0.0,0.0,24909.4500
3,NSW4,1,27,11,1,0,0,1,0.0,0.0,7506.1460
4,NSW5,1,33,8,1,0,0,1,0.0,0.0,289.7899


## Question 1: Propensity score matching

### 1. A naive analysis
Compare the distribution of the outcome variable (re78) between the two groups (if subject participated in job training or not), using plots and numbers. To summarize and compare the distributions, you may use the techniques we discussed in lectures 4 ("Read the stats carefully") and 6 ("Data visualization").

What might a naive "researcher" conclude from this superficial analysis?

### 2. A closer look at the data

### 3. A propensity score model

### 4. Balancing the dataset via matching

### 5. Balancing the groups further

### 6. A less naive analysis

## Question 2: Applied ML

### Part 1

### Part 2